# Setup and Load dataset



In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Feb 14 06:48:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    51W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [2]:

!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 18.3 MB/s eta 0:00:00


In [2]:
#!pip install transformers

In [3]:
!pip install sentence_transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 89.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 98.9 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=bd9e4c1e8ff196e13b94d9cd4956a79512141860105c10e5d32b8ef6ca20184e
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [4]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [19]:
!gdown --id 1_ila0qFpe28YFFp6-drABuq-eHutzfOz

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1_ila0qFpe28YFFp6-drABuq-eHutzfOz
To: /content/wiki_movie.zip
100% 31.4M/31.4M [00:00<00:00, 60.8MB/s]


In [20]:
!unzip -o "wiki_movie.zip"  -d  "/content"

Archive:  wiki_movie.zip
  inflating: /content/wiki_movie_plots_deduped.csv  


## Data Loading

In [21]:
from datetime import datetime
import os
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from scipy import sparse
from scipy.sparse import csc_matrix
from sklearn.decomposition import TruncatedSVD

np.random.seed(0)

#### Name of the file which contain all the item properties

In [22]:
file="wiki_movie_plots_deduped.csv"

###### Run below cell

In [23]:
content_df  = pd.read_csv( file)

In [24]:
content_df=content_df[content_df['Release Year']>2015]

## Details about dataset

In [25]:
itemid="Title"

In [26]:
features=['Plot']

In [27]:
allcols=[itemid]
for i in features:
  allcols.append(i)

# Setup

In [28]:
content_df['NewTag']=""
for i in features:
  content_df[i] = content_df[i].fillna(' ')
for i in features:
  content_df['NewTag']+=(" "+content_df[i])
content_df['NewTag']=content_df['NewTag'].astype(str)

In [29]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [30]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

In [31]:
def clean_text(text):
    
    text = text.lower()  # lowercase text
    # replace the matched string with ' '
    text = re.sub( re.compile("\'s"), ' ', text)
    text = re.sub(re.compile("\\r\\n"), ' ', text)
    text = re.sub(re.compile(r"[^\w\s]"), ' ', text)
    return text

In [32]:
stopwords=set(stopwords.words('english'))

In [33]:
def tokenizer(sentence, min_words=4, max_words=200, stopwords=stopwords, lemmatize=True):
    
    if lemmatize:
        stemmer = WordNetLemmatizer()
        tokens = [stemmer.lemmatize(w) for w in word_tokenize(sentence)]
    else:
        tokens = [w for w in word_tokenize(sentence)]
    token = [w for w in tokens if (len(w) > min_words and len(w) < max_words
                                                        and w not in stopwords)]
    return tokens    

In [34]:
content_df['clean'] = content_df['NewTag'].apply(clean_text)
# content_df['token_lem_sentence'] = content_df['clean'].apply(
#         lambda x: tokenizer(x))

# Model train

In [48]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')


In [49]:
import torch
#Select the device

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Dense({'in_features': 768, 'out_features': 768, 'bias': False, 'activation_function': 'torch.nn.modules.linear.Identity'})
)

In [50]:
corpus_embeddings = model.encode(content_df.clean.values,show_progress_bar=True, convert_to_numpy=True)

Batches:   0%|          | 0/48 [00:00<?, ?it/s]

# Search with FAISS

In [51]:
import faiss
from sentence_transformers import SentenceTransformer
encoded_data = corpus_embeddings
encoded_data = np.asarray(encoded_data.astype('float32'))
index = faiss.IndexIDMap(faiss.IndexFlatIP(corpus_embeddings.shape[1]))
index.add_with_ids(encoded_data, np.array(range(0, len(content_df))))
faiss.write_index(index, 'item.index')

### fetch details of the document

In [55]:

def search(query, k, index, model):
    qv = model.encode([query])
    top_k = index.search(qv, k)
    top_k_ids = top_k[1].tolist()[0]
    top_k_ids = list(np.unique(top_k_ids))
    results =  [content_df.iloc[idx][itemid] for idx in top_k_ids]
    return results

In [56]:
from pprint import pprint
query="movie about action and drama"
results=search(query, k=10, index=index, model=model)
content_df[content_df[itemid].isin(results)][allcols]

,Title,Plot
18037,Dhaka Attack,"The story centers on the Dhaka Metropolitan Police (DMP) Elite Forces' fierce operation against a terrorist organization, the attack is part of a plot by few foreign intelligence agencies to destabilize the country, in order to destroy the country's defense layers.[5] The film stars Arifin Shuvo as the Assistant Deputy commissioner of DMP, and the in-charge of Bomb Disposal Unit. Sumon played the role of a commander of SWAT Force. The film also features Afzal Hossain as the DMP Commissioner, Shatabdi Wadud as Intelligence officer, and Mahiya Mahi as a news reporter.[6][7]"
27538,Sa...,"SA Kannada Movie is a thriller action cinema starring Vijay Suriya , Karthik Jayaram, Samyuktha Hornad, Hemanth Hegde, Directed by Hemanth Hegde. The story takes place in Coorg in the background of a Home stay. A personal conflict in the family turns to be serious when an external intruder enter the family. The story has very unusual touch of terror and thriller. How ever the forte of the filmmaker is his screenplay and it is well depicted in this movie too. It was in Story for the controversial Poster that was set in Bangalore [1]"
27546,Mandya to Mumbai,"The movie begins in Mandya where the Hero leads a happy life with his parents. A shocking incident changes his life. His parents are murdered by an anti-social and Hero is forced to go to prison, where he is tortured. He comes across four criminals in the prison. These hardcore criminals break loose from the prison and help Hero take revenge on his parent's killers. They decide to go to Mumbai where they plan to become big gangsters. But fate has other plans."
27563,Tiger,The movie Tiger is a Kannada action packed thriller film led by the young star Pradeep (protagonist) who dreams to become a police officer which is against the wish of his father played by the well known actor and also a very famous Ex IAS officer K Shivram. Why is the father against the wish and the dream of his son is the crux of the story.
28608,Marupadi,"The film is based on a real life incident which took place in North India and it portrays the story of a family which is permanently chased by the law and power. The film is centered around the family of Aby (Rahman), a bank employee, his spouse Sara (Bhama), an orphan and their teenaged daughter Riya (Baby Nayanthara). While leading a peaceful life, Aby gets a punishment transfer to Kolkata due to some issues in the bank he worked. On their journey to the new residence at Kolkata, the anti-narcotic squad finds some drugs in their car, which led them to jail, where Sara and Riya faces sexual onsets from the police officers and inmates.[3][4] The film points finger against the prevailing social and legal systems which makes the life of innocent people tragic. Moreover, it picturize the pathetic conditions of women and children in the present day society.[5][6]"
28619,Aakashathinum Bhoomikkumidayil,The movie revolves around a village in northern Kerala where political murders happen and in such one incident a guy is murdered mistaking him to be someone else. The families of both are left stranded and the film takes us through the incidents that happen to the children of both families.
28657,Tharangam,"The movie tells about Pappan (Tovino Thomas) a cop along with the struggles led with Joy (Balu Varghese). They land in trouble while investigating a business man's wife Ommana (Neha Iyer). They cross path with a smuggler Siju (Saiju Kurup), a dreaded gangster Raghu (Unni Mukundan) which leads to a chaotic situation."
28805,Aadesh - Power Of Law,Film is a Journey of Public Prosecutor and it is a court drama.
31460,Yaanum Theeyavan,"This movie claims to represent a number of true events that have actually happened. Put together under one shell with imaginative screenplay and characters, the story revolves around 3 primary characters Michael (Ashwin Jerome), Soumya (Varsha) and Pasupathy (Raju Sundaram). The plot forms around how these 3 characters

In [57]:
from pprint import pprint
query="a movie about robots"
results=search(query, k=10, index=index, model=model)
content_df[content_df[itemid].isin(results)][allcols]

Title  \
17937                                                    Scare Campaign   
22892                                      The Thousand Faces of Dunjia   
27538                                                             Sa...   
27543                                                    Mukunda Murari   
31384                                                           Uriyadi   
33850  Tantei Opera Milky Holmes the Movie: Milky Holmes' Counterattack   
33960                         Kamen Rider Ex-Aid the Movie: True Ending   
34058                                    The Farm: En Veettu Thottathil   
34261                                              The Heritage of Love   
34276                                                        Attraction   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [58]:
from pprint import pprint
query="a movie about Artificial Intelligence"
results=search(query, k=10, index=index, model=model)
content_df[content_df[itemid].isin(results)][allcols]

Title  \
17286                  Armed Response   
17327                        Geostorm   
21613                  The White King   
22830  I Love That Crazy Little Thing   
23878        Barcelona: A Love Untold   
27538                           Sa...   
28878                          Vaapsi   
31489             Chennai 2 Singapore   
34261            The Heritage of Love   
34276                      Attraction   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

#Query generation

In [ ]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer, AutoModel
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('BeIR/query-gen-msmarco-t5-large-v1')
model = T5ForConditionalGeneration.from_pretrained('BeIR/query-gen-msmarco-t5-large-v1')
model.eval()

#Select the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (d

In [ ]:
content_df['clean']=content_df['clean'].astype(str)

In [ ]:
import tqdm

# Parameters for generation
batch_size = 5 #Batch size
num_queries = 5 #Number of queries to generate for every paragraph
max_length_paragraph = 512 #Max length for paragraph
max_length_query = 64   #Max length for output query

paragraphs = list( content_df['clean'].values)
def _removeNonAscii(s): return "".join(i for i in s if ord(i) < 128)
with open('generated_queries_all.tsv', 'w') as fOut:
    for start_idx in range(0, len(content_df['clean'].values), batch_size):
        sub_paragraphs =paragraphs[start_idx:start_idx+batch_size]
        inputs = tokenizer.prepare_seq2seq_batch(sub_paragraphs, max_length=max_length_paragraph, truncation=True, return_tensors='pt').to(device)
        outputs = model.generate(
            **inputs,
            max_length=max_length_query,
            do_sample=True,
            top_p=0.95,
            num_return_sequences=num_queries)

        for idx, out in enumerate(outputs):
            query = tokenizer.decode(out, skip_special_tokens=True)
            query = _removeNonAscii(query)
            para = sub_paragraphs[int(idx/num_queries)]
            para = _removeNonAscii(para)
            fOut.write("{}\t{}\n".format(query.replace("\t", " ").strip(), para.replace("\t", " ").strip()))

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3707: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


#Finetune

In [6]:
""
from sentence_transformers import SentenceTransformer, InputExample, losses, models, datasets
import os


train_examples = []
with open('generated_queries_all.tsv') as fIn:
    for line in fIn:
        query, paragraph = line.strip().split('\t', maxsplit=1)
        train_examples.append(InputExample(texts=[query, paragraph]))

# To use MultipleNegativesRankingLoss, it is important
# that the batch does not contain duplicate entries, i.e.
# no two equal queries and no two equal paragraphs.
# To ensure this, we use a special data loader
train_dataloader = datasets.NoDuplicatesDataLoader(train_examples, batch_size=32)

# Now we create a SentenceTransformer model from scratch
word_emb = models.Transformer('distilbert-base-uncased')
pooling = models.Pooling(word_emb.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_emb, pooling])

# MultipleNegativesRankingLoss requires input pairs (query, relevant_passage)
# and trains the model so that is is suitable for semantic search
train_loss = losses.MultipleNegativesRankingLoss(model)


#Tune the model
num_epochs = 3
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=num_epochs, warmup_steps=warmup_steps, show_progress_bar=True)

os.makedirs('output', exist_ok=True)
model.save('output/programming-model')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/238 [00:00<?, ?it/s]

Iteration:   0%|          | 0/238 [00:00<?, ?it/s]

Iteration:   0%|          | 0/238 [00:00<?, ?it/s]

search

In [10]:

from sentence_transformers import SentenceTransformer, util
import gzip
import json
import os

# Load the model we trained in 2_programming_train_bi-encoder.py
model = SentenceTransformer('output/programming-model')



In [8]:

# # Load the corpus
# docs = []
# corpus_filepath = 'wiki-programmming-20210101.jsonl.gz'
# if not os.path.exists(corpus_filepath):
#     util.http_get('https://sbert.net/datasets/wiki-programmming-20210101.jsonl.gz', corpus_filepath)

# with gzip.open(corpus_filepath, 'rt') as fIn:
#     for line in fIn:
#         data = json.loads(line.strip())
#         title = data['title']
#         for p in data['paragraphs']:
#             if len(p) > 100:    #Only take paragraphs with at least 100 chars
#                 docs.append((title, p))

# paragraph_emb = model.encode([d[1] for d in docs], convert_to_tensor=True)

# print("Available Wikipedia Articles:")
# print(", ".join(sorted(list(set([d[0] for d in docs])))))

  0%|          | 0.00/173k [00:00<?, ?B/s]

Available Wikipedia Articles:
Assembly language, C (programming language), C Sharp (programming language), C++, Go (programming language), Java (programming language), JavaScript, Keras, Laravel, MATLAB, Matplotlib, MongoDB, MySQL, Natural Language Toolkit, NumPy, PHP, Pandas (software), Perl, PostgreSQL, PyTorch, Python (programming language), R (programming language), Rust (programming language), Scala (programming language), SciPy, Scikit-learn, Swift (programming language), TensorFlow, Vue.js


In [35]:
corpus_embeddings = model.encode(content_df.clean.values,show_progress_bar=True, convert_to_numpy=True)

Batches:   0%|          | 0/48 [00:00<?, ?it/s]

semantic search

In [37]:
query = "movie about action and drama"
query_emb = model.encode(query, convert_to_tensor=True)
hits = util.semantic_search(query_emb, corpus_embeddings, top_k=10)[0]
all_hits_ids = set([hit['corpus_id'] for hit in hits])

In [40]:
pd.set_option('display.max_colwidth', None)
itemids=[]
for i in all_hits_ids:
  itemids.append(content_df.iloc[i][itemid])
content_df[content_df[itemid].isin(itemids)][allcols]

,Title,Plot
22809,Brothers,The film centers on the relationship between two brothers who used to have a bond for life but it is ripped apart through a civil war in 1936.
27064,Laali Ki Shaadi Mein Laaddoo Deewana,"The movie revolves around a royal marriage and quirky characters, who are trapped in their own tragedies."
27540,Sipaayi,"The protagonist, a TV reporter fights to stop illegal activities around the city."
27546,Mandya to Mumbai,"The movie begins in Mandya where the Hero leads a happy life with his parents. A shocking incident changes his life. His parents are murdered by an anti-social and Hero is forced to go to prison, where he is tortured. He comes across four criminals in the prison. These hardcore criminals break loose from the prison and help Hero take revenge on his parent's killers. They decide to go to Mumbai where they plan to become big gangsters. But fate has other plans."
27553,Chowka,"The film is the narrative which emerges through five different timelines and geographies within the state of Karnataka, encapsulating the lives of four people with a common friend and their love lives while sending across a strong message relating to the common man. Four individuals, each with their own dreams and aspirations, get framed for sins they did not commit and land up in jail. They meet a school teacher who not just shows them hope but also stirs up a passion towards reform.\r\nThe story takes place in the cities of Bangalore in 1986, followed by 1995 in Mysore, 2000 in Mangalore and 2007 in Bijapur. The present day is depicted in 2017, in the Central Jail of Bellary and ends in the Suvarna Vidhana Soudha premises in Belagavi. It ends on a happy but tragic note as all five are proved innocent, but the four heroes get shot by police personnel."
28619,Aakashathinum Bhoomikkumidayil,The movie revolves around a village in northern Kerala where political murders happen and in such one incident a guy is murdered mistaking him to be someone else. The families of both are left stranded and the film takes us through the incidents that happen to the children of both families.
28647,Lechmi,"The main theme of the movie revolves around four bachelors, Vineeth, Ikku, Stephen and Sudhi. The story advances on an incident that made unforeseen, dramatic changes in their lives. Once a spirit comes to their flat. This Spirit (Lechmi) doesn't remember anything, where she comes from, her real name, who killed her, and so on. With the help of Baba Swami (Biju Sopanam), an exorcist they came to know more about the spirit. The story of the film revolves around the incidents happen after it."
28652,Parava,"The story revolves in Fort Cochin about the life of two tween boys. After a half hour, it changes to the life of a young boy his mentor and his friends, of which at the end a tragedy happens. After that the story comes back to the two boys, and at the end, revenge is taken on the tragedy and the movie has a feel-good happy ending with everything back to normal."
28797,Gaon Thor Pudhari Chor,"The film’s screenplay does not spare the ruling party as well as opposition party, presenting their gimmicks to remain in limelight, while pulling each other’s legs. In short, all the politicians will be target in this film. And the makers of this film are confident that their film would offer total entertain to the Marathi audience .[5]"
28805,Aadesh - Power Of Law,Film is a Journey of Public Prosecutor and it is a court drama.


In [44]:
query = "a movie about Artificial Intelligence"
query_emb = model.encode(query, convert_to_tensor=True)
hits = util.semantic_search(query_emb, corpus_embeddings, top_k=10)[0]
all_hits_ids = set([hit['corpus_id'] for hit in hits])

In [45]:
pd.set_option('display.max_colwidth', None)
itemids=[]
for i in all_hits_ids:
  itemids.append(content_df.iloc[i][itemid])
content_df[content_df[itemid].isin(itemids)][allcols]

,Title,Plot
17211,Ghost in the Shell,"In the near future, humans are augmented with cybernetic improvements to traits such as vision, strength, and intelligence. Augmentation developer Hanka Robotics establishes a secret project to develop an artificial body, or ""shell"", that can integrate a human brain rather than an AI. Mira Killian, the sole survivor of a cyberterrorist attack which killed her parents, is chosen as the test subject after her body is damaged beyond repair. Over the objections of her designer, Dr. Ouelet, Hanka CEO Cutter decides to use Killian as a counter-terrorism operative.\r\nA year later, Killian has attained the rank of Major in the anti-terrorist bureau Section 9, working alongside operatives Batou and Togusa under Chief Daisuke Aramaki. Killian, who experiences hallucinations that Ouelet dismisses as glitches, is troubled by how little she remembers of her past. The team thwarts a terrorist attack on a Hanka business conference, and Killian destroys a robotic geisha after it murders a hostage. After learning that the geisha was hacked by an unknown entity known as Kuze, Killian breaks protocol and ""dives"" into its AI for answers. The entity attempts a counter-hack, and Batou is forced to disconnect her. They trace the hacker to a yakuza nightclub, where they are lured into a trap. The explosion destroys Batou's eyes and damages Killian's body. Cutter is enraged by Killian's actions, and threatens to have Section 9 shut down unless Aramaki keeps her in line.\r\nKuze tracks down Section 9's Hanka consultant, Dr. Dahlin, and kills her. The team links her murder to the deaths of other senior company researchers and realize that Ouelet is the next target. Kuze takes control of two sanitation workers and sends them to kill Ouelet. Batou, now with cybernetic eyes, kills one while the repaired Killian subdues the other. While they interrogate the worker, Kuze speaks through him before compelling him to commit suicide. Togusa traces the hack to a secret location, where the team discovers a large number of humans mentally linked as a makeshift signal network. Killian is captured and Kuze reveals that he is a failed Hanka test subject from the same project that created Killian. He urges her to question her own memories, then frees her and escapes.\r\nKillian confronts Ouelet, who admits that 98 test subjects died before Killian, and that her memories are implanted. Cutter has decided that Killian is a liability and orders Ouelet to kill her after she returns to Hanka Robotics. Instead, Ouelet gives Killian an address and helps her escape. Cutter kills Ouelet, but blames Killian, saying that she has gone rogue. He informs Aramaki and the team that Killian must be terminated.\r\nKillian follows the address to an apartment occupied by a widowed mother, who reveals that her daughter, Motoko Kusanagi, ran away from home a year ago and was arrested. Motoko took her own life while in custody. Killian leaves and contacts Aramaki, who allows Cutter to remotely eavesdrop on their conversation. Batou, Togusa, and Aramaki eliminate Cutter's men trying to ambush them, while Killian follows her memories to the hideaway where Motoko was last seen. There, she and Kuze meet and recall their past lives as anti-augmentation radicals who were abducted by Hanka as test subjects.\r\nCutter deploys a ""spider-tank"" to kill them. Kuze nearly dies before Killian is able to tear off the tank's motor, losing an arm in the process. Mortally wounded, Kuze offers to merge his ""ghost"" with Killian's, but Killian refuses. Kuze fades out, then a Hanka sniper kills him. Batou and the team rescue Killian, while Aramaki executes Cutter with Killian's consent. The next day, Killian, now repaired and embracing her true identity as the Japanese Motoko, reconnects with her mother and returns to work with Section 9."
17286,Armed Response,"A team of trained operatives find themselves trapped inside an isolated military compound after its artific

In [46]:
query = "a movie about robots"
query_emb = model.encode(query, convert_to_tensor=True)
hits = util.semantic_search(query_emb, corpus_embeddings, top_k=10)[0]
all_hits_ids = set([hit['corpus_id'] for hit in hits])

In [47]:
pd.set_option('display.max_colwidth', None)
itemids=[]
for i in all_hits_ids:
  itemids.append(content_df.iloc[i][itemid])
content_df[content_df[itemid].isin(itemids)][allcols]

Title  \
17211                                                     Ghost in the Shell   
17286                                                         Armed Response   
21592                                                                 Native   
22410                                                    Operation Avalanche   
22892                                           The Thousand Faces of Dunjia   
27059                                                                Machine   
28656                                                          Sherlock Toms   
33864  Terra Formars and Nicholas and Gordon’s Stranger Corps: The Fake Wave   
33890             GANTZ: O and Morrison Steel: The Greatest Years of My Life   
33896              Galactic Armored Fleet Majestic Prince: Genetic Awakening   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [41]:
!zip -r /content/model.zip /content/output

  adding: content/output/ (stored 0%)
  adding: content/output/programming-model/ (stored 0%)
  adding: content/output/programming-model/sentence_bert_config.json (deflated 4%)
  adding: content/output/programming-model/config.json (deflated 44%)
  adding: content/output/programming-model/tokenizer_config.json (deflated 41%)
  adding: content/output/programming-model/tokenizer.json (deflated 71%)
  adding: content/output/programming-model/vocab.txt (deflated 53%)
  adding: content/output/programming-model/1_Pooling/ (stored 0%)
  adding: content/output/programming-model/1_Pooling/config.json (deflated 47%)
  adding: content/output/programming-model/modules.json (deflated 53%)
  adding: content/output/programming-model/config_sentence_transformers.json (deflated 26%)
  adding: content/output/programming-model/README.md (deflated 58%)
  adding: content/output/programming-model/special_tokens_map.json (deflated 42%)
  adding: content/output/programming-model/pytorch_model.bin (deflated 8%